# Computational Social Science Project #2 

*Group number:* 7

*Group members:* Austin Biehl, Peter Soyster, Chase Stokes

*Semester:* Fall 2022

Below we fill in some of the code you might use to answer some of the questions. Here are some additional resources for when you get stuck:
* Code and documentation provided in the course notebooks  
* [Markdown cheatsheet](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) to help with formatting the Jupyter notebook
* Try Googling any errors you get and consult Stack Overflow, etc. Someone has probably had your question before!
* Send KQ a pull request on GitHub flagging the syntax that's tripping you up 

## 1. Introduction/Setup

#### a) Import relevant libraries
Add the other libraries you need for your code below and/or as you go. 

In [ ]:
# import libraries you might need here 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# use random seed for consistent results 
np.random.seed(273)

#### b) Read in and inspect data frame 
Read in the data frame and look at some of its attributes. 

In [ ]:
diabetes = pd.read_csv("Diabetes with Population Info by County 2017.csv", 
                       #CountyFips needs to be a string so leading 0 isn't dropped (this is only if you want to make choropleth map): 
                       dtype={"CountyFIPS": str}) 
diabetes.head(5)

FileNotFoundError: ignored

In [ ]:
# look at the dimensions of the diabetes data frame
print('shape: ', diabetes.shape) 

In [ ]:
pd.set_option('display.max_rows', 100) # tells pandas how many rows to display when printing so results don't get truncated

# look at the data types for each column in diabetes df 
print('data types:', diabetes.dtypes)

Immediately, we see that some of the features that should be numeric (e.g., Diabetes_Number, Obesity_Number,  and Physical_Inactivity_Number) are not. We can check to see what the non-numeric values are in a column where we are expecting numeric information with a combination of `str.isnumeric()` and `unique()`.

In [ ]:
# Return rows where the column "Diabetes_Number" is non-numeric and get the unique values of these rows
# the "~" below in front of diabetes negates the str.isnumeric() so it only takes non-numeric values
print(diabetes[~diabetes["Diabetes_Number"].str.isnumeric()]["Diabetes_Number"].unique()) 

In [ ]:
# Now do the same as above, but for "Obesity_Number" :
print(diabetes[~diabetes["Obesity_Number"].str.isnumeric()]["Obesity_Number"].unique()) 

The values contained in the two columns above making them objects (rather than integers) appear to be strings like "No Data" and "Suppressed." Let's drop those rows in the next section, and also recode Physical_Inactivity_Number to be an integer. 

#### c. Recode variables

Convert 'Diabetes_Number', 'Obesity_Number', and 'Physical_Inactivity_Number' to integers below so we can use them in our analysis. Also fill in the object type we want to recode 'sex and age_total population_65 years and over_sex ratio (males per 100 females)' to. 

In [ ]:
# Now do the same as above, but for "Physical_Inactivity_Number" :
print(diabetes[~diabetes["Physical_Inactivity_Number"].str.isnumeric()]["Physical_Inactivity_Number"].unique()) 

In [ ]:
# Diabetes
# keep only useful info about our target feature, i.e., where diabetes_number not = 'Suppressed'
diabetes = diabetes[diabetes['Diabetes_Number']!="Suppressed"]  # note that the inside reference to the diabetes df identifies the column, and the outer calls specific rows according to a condition 

# use the astype method on Diabetes_Number to convert it to an integer...if you are not sure, what does the astype() documentation tell you are possible arguments? 
diabetes['Diabetes_Number'] = diabetes['Diabetes_Number'].astype("int64") 

# Obesity
# keep only useful info about our target feature, i.e., where obesity_number not = 'No Data'
diabetes = diabetes[diabetes['Obesity_Number']!="No Data"] 

# use the astype method on Obesity_Number to convert it to an integer.
diabetes['Obesity_Number'] = diabetes['Obesity_Number'].astype("int64") 

# Physical Inactivity
# keep only useful info about our target feature, i.e., where Physical_Inactivity_Number not = 'No Data'
diabetes = diabetes[diabetes['Physical_Inactivity_Number']!="No Data"] 

# use the astype method on Physical_Inactivity_Number to convert it to an integer.
diabetes['Physical_Inactivity_Number'] = diabetes['Physical_Inactivity_Number'].astype("int64") 

# 65+ sex ratio had one "-" in it so let's drop that row first
diabetes = diabetes[diabetes['sex and age_total population_65 years and over_sex ratio (males per 100 females)']!= "-"]
# change to numeric (specifically, integer or float?) from string (because originally included the "-" )
diabetes['sex and age_total population_65 years and over_sex ratio (males per 100 females)'] = diabetes['sex and age_total population_65 years and over_sex ratio (males per 100 females)'].astype(float)


We should probably scale our count variables to be proportional to county population. We create the list 'rc_cols' to select all the features we want to rescale, and then use the `.div()` method to avoid typing out every single column we want to recode. 

In [ ]:
# select count variables to rc to percentages; make sure we leave out ratios and our population variable b/c these don't make sense to scale by population
rc_cols = [col for col in diabetes.columns if col not in ['County', 'State', 'CountyFIPS', 
                                                        'sex and age_total population_65 years and over_sex ratio (males per 100 females)', 'sex and age_total population_sex ratio (males per 100 females)', 'sex and age_total population_18 years and over_sex ratio (males per 100 females)',  
                                                        'race_total population']]
           
diabetes[rc_cols] = diabetes[rc_cols].apply(pd.to_numeric, errors='coerce') # recode all selected columns to numeric

# divide all columns but those listed above by total population to calculate rates
diabetes[rc_cols] = diabetes[rc_cols].div(diabetes['race_total population'], axis=0)


Let's check our work. Are all rates bounded by 0 and 1 as expected? 

In [ ]:
pd.set_option('display.max_columns', None)
# inspect recoded values
diabetes_summary = diabetes.describe().transpose() # note we use the transpose method rather than .T because this object is not a numpy array
  
# check recoding 
with pd.option_context('display.max_rows', 100, 'display.max_columns', None): 
    display(diabetes_summary.iloc[ : ,[0,1,3,7]]) # select which columns in the summary table we want to present

#### d. Check for duplicate columns

There are a lot of columns in this data frame. Let's see if there are any are duplicates. 

In [ ]:
# I used Google to figure this out, and adapted this example for our purposes:  
# source: https://thispointer.com/how-to-find-drop-duplicate-columns-in-a-dataframe-python-pandas/ 
def getDuplicateColumns(df):
    '''
    Get a list of duplicate columns.
    It will iterate over all the columns in dataframe and find the columns whose contents are duplicate.
    :param df: Dataframe object
    :return: List of columns whose contents are duplicates.
    '''
    duplicateColumnNames = set()
    # Iterate over all the columns in dataframe
    for x in range(df.shape[1]):
        # Select column at xth index.
        col = df.iloc[:, x]
        # Iterate over all the columns in DataFrame from (x+1)th index till end
        for y in range(x + 1, df.shape[1]):
            # Select column at yth index.
            otherCol = df.iloc[:, y]
            # Check if two columns at x 7 y index are equal
            if col.equals(otherCol):
                duplicateColumnNames.add(df.columns.values[y])
    return list(duplicateColumnNames)

duplicateColumnNames = list(getDuplicateColumns(diabetes))
print('Duplicate Columns are as follows: ')
duplicateColumnNames

In [ ]:
# now drop list of duplicate features from our df using the .drop() method
diabetes = diabetes.drop(columns=duplicateColumnNames) 

## 2. Exploratory Data Analysis

In [ ]:
# insert your EDAs and interpretations in this section 

In [ ]:
plt.scatter(diabetes.Diabetes_Number, diabetes.Obesity_Number, 
            c=diabetes.Physical_Inactivity_Number, cmap='plasma')
cbar = plt.colorbar()
cbar.set_label("Proportion of Inactive Population")
plt.xlabel("Rate of Diabetes")
plt.ylabel("Proportion of Obese Population")

**Austin**: Diabetes appears to be positively associated with obesity. As the proportion of the obese population of a county increases, so too does the rate of diabetes. Additionally, inactivity appears to also correlate with these variables. From the figure, we can see that in counties with high rates of obesity and diabetes also tend to have high rates of inactivity. Taken together, we should expect that both obesity and inactivity will be important predictors of a county's rate of diabetes

**Peter's EDA***

In [ ]:
39
# uncomment to install required packages
#!pip install -U plotly
#!pip install -U dash
#!pip install -U kaleido

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
    
# This file contains geometry information for US counties, with corresponding FIPS code as column 'id'
counties["features"][0]

In [ ]:
# import packages for plotting
import kaleido
import plotly.express as px
import plotly.io as pio

In [ ]:
fig = px.choropleth(diabetes, geojson= counties, locations='CountyFIPS', color='Physical_Inactivity_Number',
                           color_continuous_scale="Viridis",
                           range_color=(0, 0.2),
                           scope="usa",
                           labels={'Physical Inactivity Number Number':'Rate of Physical Inactivity',
                                  'CountyFIPS': 'County ID'})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

**Peter**: Based on the chloropleth figure, there appears to be a relationship between geography and the physical activity. In general, there are higher rates of physical inactivity in the southeastern states of the country, including Louisiana, Mississippi, Alabama, Georgia, South Carolina, and Florida. The figure suggests that community-wide factors, such as culture and social networks, may have an influence upon the rate of physical inactivity in addition to individual-level factors, such as age and physical inactivity.

In [ ]:
# chase EDA
j_plot = sns.jointplot(x='Diabetes_Number', y='sex and age_total population_male', data = diabetes, kind = "reg")
sns.regplot(x='Diabetes_Number', y='sex and age_total population_male', data=diabetes)
j_plot.set_axis_labels('Rate of Diabetes', 'Proportion of Population Male', fontsize=16)
plt.tight_layout();

In [ ]:
filter_col = ['sex and age_total population_male',
              'sex and age_total population_female',
              'sex and age_total population_18 years and over_male',
              'sex and age_total population_18 years and over_female',
              'sex and age_total population_65 years and over_male',
              'sex and age_total population_65 years and over_female']

plt.bar(x = filter_col, height = diabetes[filter_col].mean())
plt.xticks(filter_col, rotation = 90);

**Chase**: Diabetes does not appear to be associated with gender. In particular, most counties appear to have a relatively even gender split, except for a few outliers. The rate of diabetes does not increase nor decrease signficantly with this attribute. The chart above shows a straight line of fit, likely due in part to the even spread of gender between counties. Regardless of the reason, the variables relating to gender will likely not be strong predictors for the rate of diabetes in a given county.

## 3. Prepare to Fit Models

### 3.1 Finalize Data Set

We've already cleaned up the data, but we can make a few more adjustments before partitioning the data and training models. Let's recode 'State' to be a categorical variable using `pd.get_dummies` and drop 'County' using `.drop()` because 'CountyFIPS' is already a unique identifier for the county. 

In [ ]:
# create dummy features out of 'State' , which might be related to diabetes rates 
diabetes_clean = pd.get_dummies(diabetes, 
                               columns = ['State'],  
                               drop_first = True) # only create 49 dummies by dropping first in category

# drop 'County' variable
diabetes_clean = diabetes_clean.drop(labels = ['County', 'CountyFIPS'],
                               axis = 1) # which axis tells python we want to drop columns rather than index rows?

# look at first 10 rows of new data frame 
diabetes_clean.head(10) 

### 3.2/3.3 Partition Data and Feature Selection

Now, we will partition our data to prepare it for the training process. We will use 60% train—20% validation—20% test in this case. More data in the training set lowers bias, but then increases variance in the validation/test sets. Balancing between bias and variance with choice of these set sizes is important as we want to ensure that there is enough data to train on to get good predictions, but also want to make sure our hold-out sets are representative enough.

In [ ]:
# drop gender based interaction features since not indicated useful in Chase EDA
diabetes_clean = diabetes_clean.drop(filter_col, axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split

# Set y 
y = diabetes_clean['Diabetes_Number']

# X (everything except diabetes, our target)
X_unprocessed = diabetes_clean.drop(labels = ['Diabetes_Number'], axis = 1)
X_unprocessed

We should also preprocess our data. Using the `preprocessing` module from sklearn, let's scale our features so that they are mean-centered.

In [ ]:
X_unprocessed.shape

In [ ]:
from sklearn import preprocessing

X = preprocessing.scale(X_unprocessed)
X = pd.DataFrame(X, columns = X_unprocessed.columns)

We can also get rid of the 0 variance features using the `VarianceThreshold()` method from `feature_selection`. 

In [ ]:
from sklearn import feature_selection

def variance_threshold_selector(data, threshold=0):
    selector = feature_selection.VarianceThreshold(threshold)
    selector.fit_transform(data)
    return data[data.columns[selector.get_support(indices=True)]]


In [ ]:
X = variance_threshold_selector(X)
X.shape

And finally, let's split our data:

In [ ]:
# split the data
# train_test_split returns 4 values: X_train, X_test, y_train, y_test, so how do we create a 60-20-20 train-validate-test split? 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size = 0.2)

## 4. Train Models

In [ ]:
from sklearn.linear_model import Ridge, Lasso, LassoCV, LinearRegression

### 4.1 Model Description

#### Model 1: Linear Regression?

Explain why we picked Model 1.

#### Model 2: Ridge

Like OLS, ridge regression estimates parameters that minimize the sum of the squared residuals; however, unlike OLS, ridge regression also seeks to minimize the sum of the squared coefficients, as a function of a tuning parameter. The rationale behind ridge regression is that by shrinking the estimates toward zero, the variance of the estimated model is reduced. Moreover, ridge regression is particularly well suited to models consisting of a larger number of predictors.

#### Model 3: LASSO

Since there are many features included in the model, even after excluding based on the EDA results, the LASSO model allows us perform feature selection computationally. This will be a great help in determining which factors actually contribute to the rate of diabetes by county.

### 4.2 Train Models

#### Model 1: Linear Regression

In [ ]:
# model 1 Austin

# create a model
lin_reg = LinearRegression()

# fit the model
lin_model = lin_reg.fit(X_train, y_train)

#### Model 2: Ridge

In [ ]:
ridge_reg = Ridge() 
ridge_model = ridge_reg.fit(X_train, y_train)
ridge_reg_data = pd.DataFrame([ridge_model.coef_, X.columns]).T
ridge_reg_data.columns = ['Coefficient', 'Feature']

In [ ]:
print(ridge_model.coef_)
print(ridge_model.intercept_)

In [ ]:
figure = plt.figure()
figure.subplots_adjust(wspace = .5, hspace=.5)
figure.add_subplot(1, 2, 1)
sns.barplot(x="Coefficient", y="Feature", data=ridge_reg_data).set_title("Ridge Coefficients")
figure.add_subplot(1, 2, 2)
sns.barplot(x="Coefficient", y="Feature", data=lin_reg_data).set_title("OLS Coefficients")
plt.show()

In [ ]:
print(sum(ridge_model.coef_**2))
print(sum(lin_model.coef_**2))

NameError: ignored

#### Model 3: LASSO

In [ ]:
# model 3
# LASSO
lasso_reg = Lasso(max_iter = 10000)
lasso_model = lasso_reg.fit(X_train, y_train)

## 5. Validate and Refine Models

In [ ]:
#create function to calculate the root mean squared error
def rmse(pred, actual):
  return np.sqrt(np.mean((pred-actual)**2))

### 5.1 Predict on the Validation Set

#### Model 1: Linear Regression

In [ ]:
# predict the number of riders
lin_pred = lin_model.predict(X_validate)

# plot the residuals on a scatter plot
plt.scatter(y_validate, lin_pred)
plt.title('Linear Model (OLS) Predicted v. Actual')
plt.xlabel('actual value')
plt.ylabel('predicted value')
plt.show()

In [ ]:
#calculate the mean squared error of the linear model
rmse(lin_pred, y_validate)

The linear regression model produces a low mean squared error, indicating that the values we forecast were close to the actual values in the validation set. 

#### Model 2: Ridge

In [ ]:
# use the model to make predictions
ridge_pred = ridge_model.predict(X_validate)

# plot the predictions
plt.scatter(y_validate, ridge_pred)
plt.title('Ridge Model')
plt.xlabel('actual values')
plt.ylabel('predicted values')
plt.show()

In [ ]:
rmse(ridge_pred, y_validate)

#### Model 3: LASSO

In [ ]:
# validate model 3
# LASSO
lasso_pred = lasso_model.predict(X_validate)

plt.scatter(lasso_pred, y_validate)
plt.title('LASSO Model')
plt.xlabel('predicted values')
plt.ylabel('actual values')
plt.show()

In [ ]:
rmse(lasso_pred, y_validate)

The LASSO model shows a clear error in its predictions, despite a low RMSE of 0.027.

### 5.2 Feature Selection

#### Model 1: Linear Regression?

In [ ]:
cutoff = 0.2
X_featureselection = X.loc[:, abs(lin_model.coef_) > cutoff]

coefs = lin_model.coef_[abs(lin_model.coef_) > cutoff]
lin_reg_data = pd.DataFrame([coefs, X_featureselection.columns]).T
lin_reg_data.columns = ['Coefficient', 'Feature']

# Plot
plt.figure(figsize=(50,50))
ax = sns.barplot('Coefficient', 'Feature', data = lin_reg_data)
ax.set_title("OLS Coefficients")

plt.show()

In [ ]:
# update sets to select these features (from the linear regression)
X_train_linearselected = X_train.loc[:, lin_model.coef_ > .2]
X_validate_linearselected = X_validate.loc[:, lin_model.coef_ > .2]
X_test_linearselected = X_test.loc[:, lin_model.coef_ > .2]

#### Model 2: Ridge?

In [ ]:
param_grid = {'alpha': np.arange(.1, 1, .1),
              'fit_intercept': ['True', 'False'],
              'solver': ['auto', 'svd', 'cholesky', 'lsqr']}

ridge_grid_reg = GridSearchCV(ridge_reg, param_grid, cv=10)
ridge_grid_reg.fit(X_train, y_train)

best_index = np.argmax(ridge_grid_reg.cv_results_["mean_test_score"])
best_ridge_pred = ridge_grid_reg.best_estimator_.predict(X_validate)

print('Best CV R^2:', max(ridge_grid_reg.cv_results_["mean_test_score"]))
print('Validation R^2:', ridge_grid_reg.score(X_validate, y_validate))
print('Validation RMSE', rmse(best_ridge_pred, y_validate))

#### Model 3: LASSO

In [ ]:
np.unique(lasso_model.coef_)

As shown above, the LASSO model selected has reduced all the coefficients to 0. This means that the predicted value is the same every time. So, we must have selected a lambda value which is too high (i.e., the model default of `alpha = 1.0` is too high). To fix this, we bring in the `LassoCV()` function to select better values of alpha through cross-validation.

In [ ]:
# model 3 CV
# LASSOCV
lasso_reg = LassoCV(max_iter = 10000)
lasso_model = lasso_reg.fit(X_train, y_train)
lasso_pred = lasso_model.predict(X_validate)

plt.scatter(lasso_pred, y_validate)
plt.title('LASSO Model')
plt.xlabel('predicted values')
plt.ylabel('actual values')
plt.show()

In [ ]:
cutoff = 0.001
X_featureselection = X.loc[:, abs(lasso_model.coef_) > cutoff]

coefs = lasso_model.coef_[lasso_model.coef_ != 0]
lasso_reg_data = pd.DataFrame([coefs, X_featureselection.columns]).T
lasso_reg_data.columns = ['Coefficient', 'Feature']

figure = plt.figure()
figure.subplots_adjust(wspace = .5, hspace=.5)
figure.add_subplot(1, 2, 1)
# Plot for LASSO coefficients here
sns.barplot(x = 'Coefficient', y = 'Feature', data = lasso_reg_data)
plt.title('LASSO coefficients');

In [ ]:
lasso_model.alpha_

In [ ]:
rmse(lasso_pred, y_validate)

This RMSE, of 0.018 is lower than the original model with `alpha = 1`, which had an RMSE of 0.027.

Through cross-validation on the validation data sets, we were able to find the best alpha value, around 0.0004. This is far from the default value of 1 and likely indicates that the LASSO model adds little to a regular `LinearRegression()`. This now serves to help us with relevant feature selection, as LASSO performs feature selection as part of the modeling process. Rather than have no features to choose from, the adjustment to alpha allows us to determine key features which contribute to the predictive ability of the model.

In [ ]:
# update sets to select these features (from the lasso regression)
X_train_lassoselected = X_train.loc[:, abs(lasso_model.coef_) > cutoff]
X_validate_lassoselected = X_validate.loc[:, abs(lasso_model.coef_) > cutoff]
X_test_lassoselected = X_test.loc[:, abs(lasso_model.coef_) > cutoff]

### 5.3 Test Set

#### Chosen Model: ?

In [ ]:
# test set model

### 5.4 Implement a Cross-Validation Approach

#### Chosen Model: ?

In [ ]:
# cv model

In [ ]:
# cv model 2

## 6. Discussion Questions

In [ ]:
# insert responses for discussion Qs here

### 6.1

What is bias-variance tradeoff? Why is it relevant to machine learning problems like this one?

*Answer*: The bias-variance tradeoff refers to the fact that there is a tradeoff regarding a model's ability to minimize bias and variance. Bias relates to difference between the model we make and the actual, correct value that we are trying to predict. A model with high bias might be oversimplified, and thus have a high amount of errors on both training and test data. Variance refers to the variability of our models predictions for a given data point. Models with a high amount of variance try to really close train on the training data, and therefore often don't perform well on the test data because they are too specific to one particular condition. 

The bias-variance trade off becomes relevant when trying to create a model given a large amount of parameters, as we were with the diabetes data set. If our model is too simple, and we only put in two or three of the biggest parameters (obesity, physical activity, etc), the model might be clunky and non-specific, missing so much nuance that it is no longer useful. In contrast, if we used ALL the parameters in the data set, parameters that just happened to have small effects in this dataset, but that aren't more broadly useful, might result in us creating a model that is TOO specific, and that wouldn't help us make accurate predictions with new data.  

### 6.2

Define overfitting, and why it matters for machine learning. How can we address it?

*Answer*: Generally speaking, overfitting is when a statistical model is tuned to the characteristics of a specific dataset in a way that decreases the generalizability of the model to new data. Overfitting is particularly common when a model has a large number of independent variables relative to the number of total observations. As discussed above, sometimes by increasing a models bias (e.g., increasing the extert to which is doesn't try to perfectly predict each data point) you decrese the risk of overfitting and actually end up with more a more generalizable model.  

Overfitting is a particularly important issue in machine learning models as these types of models tend to have a large number of independent variables. To address overfitting in machine learning models, researchers often partician the data into three seperate bins "training data", "validation data", and "testing data". By fitting the model to the training data, and testing the generalizability of model performance on the validation data, researchers have the ability to tune the model in a way that maximized generalizability. Once the iterative model building process is complete, the final model is tested with the testing data, to determine the generalized performance of the model. 

### 6.3

Discuss your Analysis in 2-3 Paragraphs.

*Answer*: In this project, we used several basic machine learning models to predict diabetes rates in US counties, with the goal of developing mechanisms to provide more targeted interventions. There were several stages involved with this process, including exploring the data, creating and training the models, and then testing and selecting a final model for use. 

To begin with, each of us explored the data in order to get a sense of which variables might be most useful in the construction of a machine learning model. This was a helpful step, as the diabetes data set contained a lot of different factors that could potentially contribute to a final model. Exploratory data analysis gave our team a sense of which factors would likely be central to a final model. In this case, we created scatterplots, bar charts, and choropleths, to get a sense of the data. All our visualizations told the story that counties with higher rates of obesity, and lower rates of physical activity, tended to have higher rates of diabetes. Additionally, race seemed to be important as well, with higher diabetes rates observed in non-white populations. 

After splitting our data into training, validation, and test data, we created 3 different machine learning models:  a linear regression model, a ridge regression model, and the LASSO model. After training and validating the models, we removed variables from the coefficients from the regression models that were either 0 or near 0, in order to get rid of excess variables that were not contributing to our ability to make accurate predictions. Additionally, we used the LASSO approach to reduce the number of coefficients to 10. Interestingly, all 3 approaches resulted in very similar RMSE values. However, we ultimately decided to use the LASSO model for our final test. We thought that, while it provided only a nominal improvement in prediction accuracy, it was by far the more parsimonious model, and as such might contribute to greater interpretability by policy makers. 